Get Kiva data
=====

In [16]:
reset -fs

In [17]:
import os
import json
import logging
import requests
import time
from urllib.request import urlopen, Request

from pandas.io.json import json_normalize
import pandas as pd

In [18]:
# # # Debugging code
# for code in [None]:
#     print(code)
#     url = f'http://api.kivaws.org/v1/loans/search.json?country_code={code}&per_page=5'
#     r = requests.get(url)
#     # r.status_code
#     print(r.json()['loans'][0])
#     break

In [19]:
country_codes_without_data = ["DZ", "AO", "SH", "CV", "CF", "TD", "KM", "DJ","GQ", "ER", "SZ",  "ET",  "GA", "GM", "GN",  "GW",    "LY",  "MU", "YT", "MA",  "NE",  "ST",  "RE", "ST",   "SC",  "SH", "SD","SZ","TN",  ]

country_codes_with_data = ["BJ", "BW", "BF", "BI", "CM", "CG", "CD", "EG", 
                           "GH", "CI", "KE", "LS", "LR", "MG", "MW", "ML", 
                           "MR", "MZ", "NA", "NG", "RW", "SN", "SL", "SO",
                           "ZA", "SS", "TZ", "TG", "UG", "CD", "ZM", "TZ", "ZW"]

In [42]:
loans_complete = pd.DataFrame()


for country_iso_code in country_codes_with_data:   
#     n_pages = 1
#     for page in range(1, n_pages+1):
        print(f"Requesting - Country: {country_iso_code}, Page: {page}")
        n_per_page = 10
        url = f'http://api.kivaws.org/v1/loans/search.json?country_code={country_iso_code}&per_page={n_per_page}&page={page}'
        r = requests.get(url)
        if r.status_code == 403:
            print("Too many requests. Slow down 🐢")
        
        data = json.loads(r.text)
        if data['paging']['total'] == 0:
            break # Stop if there are no loans
            
        loans = json_normalize(data['loans'])
        
        # Take the loan id column from our retrieved loan data, and use it to pull additional details
        # about our loans. We store this in a seperate table loans_details
        loan_ids = loans['id'].tolist()
        loan_ids_str = ','.join(str(e) for e in loan_ids)

        r = requests.get('https://api.kivaws.org/v1/loans/'+loan_ids_str+'.json')
        ld_data = json.loads(r.text)
        loans_ext = json_normalize(ld_data['loans'])

        # Join on two dataframes
        temp = pd.merge(loans, 
                 loans_ext, 
                 how='inner', 
                 on='id', 
                 left_on=None, right_on=None, left_index=False, right_index=False, sort=True, suffixes=('_x', '_y'), 
                 copy=True, indicator=False, validate=None)
        loans_complete.append(temp)
         
#         print("The number of loans on current page: {}".format(len(loans_full.index)))
        
        break
        time.sleep(1) # Wait a little while to so we don't overload Kiva servers

Requesting - Country: BJ, Page: 1


In [47]:
loans_complete.head(n=2)

""


In [46]:
loans_complete.shape

(0, 0)

In [25]:
loans_details.head(n=2)

,activity,bonus_credit_eligibility,borrowers,description.languages,description.texts.en,description.texts.fr,funded_amount,funded_date,id,image.id,...,terms.loss_liability.currency_exchange,terms.loss_liability.currency_exchange_coverage_rate,terms.loss_liability.nonpayment,terms.repayment_interval,terms.repayment_term,terms.scheduled_payments,themes,translator.byline,translator.image,use
0,Agriculture,False,"[{'first_name': 'Tolaro Global', 'last_name': ...",[en],"In February 2017, MCE made a 12-month loan of ...",NaN,50000,2018-04-06T22:46:13Z,1480779,2790263,...,none,0.1,lender,At end of term,14,[],NaN,NaN,NaN,to promote the growth of the business by trans...
1,Agriculture,False,"[{'first_name': 'Tolaro Global', 'last_name': ...",[en],"<a href=""http://agpowerbenin.com/""> Tolaro Glo...",NaN,50000,2017-04-18T06:42:43Z,1266423,2479333,...,none,0.1,lender,Irregularly,12,[],NaN,NaN,NaN,to add value and jobs to the local economy by ...


In [33]:
loans_full.shape

(10, 25)

In [34]:
loans_full.head(n=2)

,activity,bonus_credit_eligibility,borrower_count,description.languages,funded_amount,id,image.id,image.template_id,lender_count,loan_amount,...,location.town,name,partner_id,planned_expiration_date,posted_date,sector,status,tags,themes,use
0,Agriculture,False,1,[en],50000,1480779,2790263,1,1574,50000,...,Parakou,Tolaro Global,478,2018-04-07T14:50:22Z,2018-03-08T15:50:22Z,Agriculture,funded,"[{'name': 'user_favorite'}, {'name': '#Vegan'}...",NaN,to promote the growth of the business by trans...
1,Agriculture,False,1,[en],50000,1266423,2479333,1,1519,50000,...,Parakou,Tolaro Global,478,2017-05-04T07:30:08Z,2017-04-04T07:30:08Z,Agriculture,funded,"[{'name': 'user_favorite'}, {'name': '#Sustain...",NaN,to add value and jobs to the local economy by ...


,activity_x,bonus_credit_eligibility_x,borrower_count,description.languages_x,funded_amount_x,id,image.id_x,image.template_id_x,lender_count_x,loan_amount_x,...,terms.loss_liability.currency_exchange,terms.loss_liability.currency_exchange_coverage_rate,terms.loss_liability.nonpayment,terms.repayment_interval,terms.repayment_term,terms.scheduled_payments,themes_y,translator.byline,translator.image,use_y
0,Retail,False,10,"[fr, en]",4825,736066,726677,1,60,4825,...,shared,0.1,lender,Monthly,8,[],NaN,Christopher Smith,NaN,
1,Food Production/Sales,False,1,"[fr, en]",225,743236,726677,1,8,225,...,shared,0.1,lender,Monthly,14,[],NaN,Lindsay Ferrara,839627.0,
2,Retail,False,1,"[fr, en]",400,743238,726677,1,10,400,...,shared,0.1,lender,Monthly,14,[],NaN,NaN,NaN,
3,Phone Use Sales,False,11,"[fr, en]",5850,744232,726677,1,176,5850,...,shared,0.1,lender,Monthly,12,[],NaN,Daniel Kuey,2835726.0,
4,Clothing Sales,False,12,"[fr, en]",1250,744237,726677,1,42,1250,...,shared,0.1,lender,Monthly,10,[],NaN,Daniel Kuey,2835726.0,
5,Cosmetics Sales,False,6,"[fr, en]",500,744240,726677,1,13,500,...,shared,0.1,lender,Monthly,10,[],NaN,Daniel Kuey,2835726.0,
6,Jewelry,False,4,"[fr, en]",1000,744243,726677,1,31,1000,...,shared,0.1,lender,Monthly,13,[],NaN,Daniel Kuey,2835726.0,
7,Crafts,False,1,[en],300,767909,1414691,1,12,300,...,none,NaN,lender,At end of term,10,[],"[Fair Trade, Innovative Loans]",NaN,NaN,to invest in a bulk purchase of raw materials ...
8,Agriculture,False,1,[en],50000,1266423,2479333,1,1519,50000,...,none,0.1,lender,Irregularly,12,[],NaN,NaN,NaN,to add value and jobs to the local economy by ...
9,Agriculture,False,1,[en],50000,1480779,2790263,1,1574,50000,...,none,0.1,lender,At end of term,14,[],NaN,NaN,NaN,to promote the growth of the business by trans...


# TODO: Refactor below this cell 

In [ ]:
raise Exception('Try 1 request and stop') 


-----

Let's save our data!

In [ ]:
path = 'data/'
loans_full.to_csv(path+'loans_full_large.csv')
loans_details.to_csv(path+'loans_details.csv')

We will also pull data about Kiva's partners, who are the intermediaries between Kiva and end recipients for the majority of loans made.

In [ ]:
data_path = '~/intro_to_machine_learning/data'
df=pd.read_csv(data_path+'/loans_details.csv', low_memory=False)
df.head()

In [ ]:
df=df[df['partner_id'].notnull()]
df['partner_id']=df['partner_id'].astype(int).astype(str)
partner_id=df['partner_id'].unique().tolist()

In [ ]:
d = r.get('https://api.kivaws.org/v1/partners/322.json?app_id=org.deltanalytics')

In [ ]:
d.headers

In [ ]:
d.json();

In [ ]:
def extract_loan_partners(partner_ids):
    loan_partner_details=pd.DataFrame()
    for n in partner_ids:
        d = r.get('https://api.kivaws.org/v1/partners/'+n+'.json?app_id=org.deltanalytics')
        data = json.loads(d.text)
        partners=json_normalize(data['partners'])
        loan_partner_details=loan_partner_details.append(partners, ignore_index=True)
    
    return loan_partner_details  

In [ ]:
loan_partner_details=extract_loan_partners(partner_id)

In [ ]:
loan_partner_details.to_csv('~/intro_to_machine_learning/data/loans_partner_details.csv')

## Pulling each loan's lender details

Now that we have selected a subset of loans to focus in on, we want to pull all of the lenders of these loans. We do so here by first creating a list of the loans we are interested in. 

In [ ]:
loan_ids = df['id'].astype(int).astype(str)
loan_ids = loan_ids.unique().tolist()
loan_ids[0:5]

Here we define a function that will extract details of every lender who has contributed to a single loan, and then loop it over the list of loans that we created in the previous step:

In [ ]:
def extract_loan_lenders(loan_ids):
    loan_lenders_details=pd.DataFrame()
    for n in loan_ids:
        d = r.get('https://api.kivaws.org/v1/loans/'+n+'/lenders.json?app_id=org.deltanalytics')
        data = json.loads(d.text)
        if len(data['lenders']) != 0:
            lenders=json_normalize(data['lenders'])
        else: lenders = pd.DataFrame()
        lenders['loan_id'] = n
        loan_lenders_details = loan_lenders_details.append(lenders, ignore_index=True)

    return loan_lenders_details  

In [ ]:
loan_lenders_details = extract_loan_lenders(loan_ids)

We then write the output to a data frame. Let's take a look at what we've got! 

In [ ]:
loan_lenders_details.head(100)

In [ ]:
loan_lenders_details.to_csv('~/intro_to_machine_learning/data/loans_lenders_details.csv')